Libraries

In [1]:
import pandas as pd
import numpy as np

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import requests
from random import randint
from time import sleep

from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler
from matplotlib.lines import Line2D

from sklearn.cluster import KMeans

import joblib

### Spotipy

In [2]:
secrets_file = open("SpotifySecret.txt","r")

In [3]:
string = secrets_file.read()

In [4]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

In [5]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['cs']))

In [ ]:
def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3))
    return tracks

I will mix several TOP song lists

In [ ]:
plIds = ["37i9dQZEVXbMDoHDwVN2tF","4NLrcCMFyUgtXor7EjlE7d","5ZyAjPmaz9KOB4f73RFYvi","6MJSGcF4iV79gyo8xZpd8U", 
         "6CfQ2Ptcxju9l6YC2LXzUb", "4hNaqkezNNv7ImufYGHgkf", "1zrx1DCawPLg6Y8AMFgCUZ", "2iBH9S3UXlrtUBxjffgZEh",
         "1cax1gYS1699tnR5bVjtiY", "2P4tjMKY8ORcUU17QBz83q", "5fcPNR6KXA3iYgNrvTDZpY", "62SVVpIhl7b5c82G0IH603"]

In [ ]:
#Combining all lists in one
all_tracks = []
for i in range(len(plIds)):
    all_tracks = all_tracks + (get_playlist_tracks(plIds[i]))

In [ ]:
title = []
artist = []
danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []


for i in range(len(all_tracks)):
    title.append(all_tracks[i]["track"]["name"])
    artist.append(all_tracks[i]['track']['artists'][0]['name'])
    song_uri = all_tracks[i]["track"]["uri"]
    audio = sp.audio_features(song_uri)
    danceability.append(audio[0]['danceability'])
    energy.append(audio[0]['energy'])
    key.append(audio[0]['key'])
    loudness.append(audio[0]['loudness'])
    mode.append(audio[0]['mode'])
    speechiness.append(audio[0]['speechiness'])
    acousticness.append(audio[0]['acousticness'])
    instrumentalness.append(audio[0]['instrumentalness'])
    liveness.append(audio[0]['liveness'])
    valence.append(audio[0]['valence'])
    tempo.append(audio[0]['tempo'])
    sleep(randint(1,3))
    
audioFeatures = pd.DataFrame({"title":title,
                        "artist":artist,
                        "danceability":danceability,
                        "energy":energy,
                        "key":key,
                        "loudness":loudness,
                        "mode":mode,
                        "speechiness":speechiness,
                        "acousticness":acousticness,
                        "instrumentalness":instrumentalness,
                        "liveness":liveness,
                        "valence":valence,
                        "tempo":tempo
                      })

In [6]:
#Importing a saved copy of the songs with their audio features which I obtained with the instructions above
audioFeatures = pd.read_csv('TOPListsLarge.csv')

In [7]:
#Droping duplicates
audioFeatures = audioFeatures.drop_duplicates(['title'], keep='first')

In [8]:
#Creating a copy
audioFeatures2 = audioFeatures.copy()

### KMeans 

In [9]:
#Splitting the 2
y = audioFeatures['title']
X_audio = audioFeatures.drop(['artist','title'], axis=1)
#Scaling X
scaler = StandardScaler()
X_prep = scaler.fit_transform(X_audio)
joblib.dump(scaler, 'scaler.bin')
#Aplying KMeans
kmeansCreated = KMeans(n_clusters=4, random_state=1234)
kmeansCreated.fit(X_prep)
clusters = kmeansCreated.predict(X_prep)
#Creating a column with the cluster from each Song
audioFeatures2['cluster'] = clusters

Now, I have the hole list:

    - "List of many songs with theis audio features clustered": audioFeatures2
    
So i will continue creating some functions to get the cluster of the song

In [10]:
def SongAudioFeat(song, audio):
    #Creating a DF with the audio Features
    SongAudioFeatures = pd.DataFrame()
    SongAudioFeatures["title"] = [song["tracks"]["items"][0]["name"]]
    SongAudioFeatures["danceability"] = [audio[0]['danceability']]
    SongAudioFeatures["energy"] = [audio[0]['energy']]
    SongAudioFeatures["key"] = [audio[0]['key']]
    SongAudioFeatures["loudness"] = [audio[0]['loudness']]
    SongAudioFeatures["mode"] = [audio[0]['mode']]
    SongAudioFeatures["speechiness"] = [audio[0]['speechiness']]
    SongAudioFeatures["acousticness"] = [audio[0]['acousticness']]
    SongAudioFeatures["instrumentalness"] = [audio[0]['instrumentalness']]
    SongAudioFeatures["liveness"] = [audio[0]['liveness']]
    SongAudioFeatures["valence"] = [audio[0]['valence']]
    SongAudioFeatures["tempo"] = [audio[0]['tempo']]
    
    return(SongAudioFeatures)

In [11]:
def cluster_for_a_Song(SongAudioFeatures, kmeans, scaler=joblib.load('scaler.bin')):
    #Splitting the DF
    y = SongAudioFeatures['title']
    X_audio = SongAudioFeatures.drop(['title'], axis=1)
    #Scaling X
    X_prep = scaler.transform(X_audio)
    #Aplying KMeans to calculate the cluster
    cluster = kmeansCreated.predict(X_prep)
    
    return (cluster)

### Recommender

I ask for a song and I get its cluster to give a recommendation

In [12]:
#Asking for a Song
print("What is your song title?")
favSong = input()

#Looking for a Recomendation
        
#If it's in the general list, I will recommend a song from its cluster    
if favSong in audioFeatures2['title'].values:
    songRow = audioFeatures2[audioFeatures2.title == favSong]
    PossSongs = audioFeatures2[audioFeatures2.cluster == int(songRow.cluster.to_string(index=False))]
    RecSong = PossSongs['title'].sample().to_string(index=False)
    print("Recommended song:", RecSong)
    #Looking for it on Spotify
    song = sp.search(q=RecSong, limit=1)
    link = song["tracks"]["items"][0]['external_urls']['spotify']
    print("You can find it here:", link)
        
#If it isn't listed i will look for it on Spotify, cluster it and recommend a song from its cluster
else: 
    #Searching for the song and its audio features in Spotify
    song = sp.search(q=favSong, limit=1)
    song_uri = song["tracks"]["items"][0]["uri"] 
    audio = sp.audio_features(song_uri)
    #Checking if it exists and it has audio features:
    if ((len(song['tracks']['items']) > 0) & (len(audio)>0)):
        #Calculating its cluster
        SongAudioFeatures = SongAudioFeat(song, audio)
        cluster = int(cluster_for_a_Song(SongAudioFeatures, kmeansCreated))
        #Recomending a song with the same cluster
        PossSongs = audioFeatures2[audioFeatures2.cluster == cluster]
        RecSong = PossSongs['title'].sample().to_string(index=False)
        print("Recommended song:", RecSong)
        #Looking for it on Spotify
        song = sp.search(q=RecSong, limit=1)
        link = song["tracks"]["items"][0]['external_urls']['spotify']
        print("You can listen it here:", link)
        
        #Looking for an artist
        RecArtist = PossSongs['artist'].sample().to_string(index=False)
        artist = sp.search(q=RecArtist, limit=1)
        print("Recommended artist:", RecArtist)
        link = artist["tracks"]["items"][0]['external_urls']['spotify']
        print("You can listen one of his songs here:", link)
    else: 
        print("Your song is not listed on Spotify or its audio features are not available")

What is your song title?
Do It to It
Recommended song: Halo
You can listen it here: https://open.spotify.com/track/4JehYebiI9JE8sR8MisGVb
Recommended artist: Outlandish
You can listen one of his songs here: https://open.spotify.com/track/5HERF1jKSRu3mMB1YTwzFC
